In [1]:
import sys
import os

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from few.trajectory.inspiral import EMRIInspiral
from few.amplitude.romannet import RomanAmplitude
from few.amplitude.interp2dcubicspline import Interp2DAmplitude
from few.waveform import FastSchwarzschildEccentricFlux, SlowSchwarzschildEccentricFlux, GenerateEMRIWaveform
from few.utils.utility import (get_overlap, 
                               get_mismatch, 
                               get_fundamental_frequencies, 
                               get_separatrix, 
                               get_mu_at_t, 
                               get_p_at_t, 
                               get_kerr_geo_constants_of_motion,
                               xI_to_Y,
                               Y_to_xI)

from few.utils.ylm import GetYlms
from few.utils.modeselector import ModeSelector
from few.summation.interpolatedmodesum import CubicSplineInterpolant
from few.waveform import SchwarzschildEccentricWaveformBase
from few.summation.interpolatedmodesum import InterpolatedModeSum
from few.summation.directmodesum import DirectModeSum
from few.utils.constants import *
from few.summation.aakwave import AAKSummation
from few.waveform import Pn5AAKWaveform, AAKWaveformBase

/home/masrukuddin/anaconda3/envs/few_env/lib/python3.12/site-packages/cupy/_environment.py:369: UserWarning: CuPy failed to preload library (/home/masrukuddin/.cupy/cuda_lib/11.x/cutensor/2.0.1/lib/libcutensor.so.2.0.1): OSError (libcublasLt.so.11: cannot open shared object file: No such file or directory)
  warnings.warn(msg)


In [2]:
use_gpu = False

# keyword arguments for inspiral generator (RunSchwarzEccFluxInspiral)
inspiral_kwargs={
        "DENSE_STEPPING": 0,  # we want a sparsely sampled trajectory
        "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    }

# keyword arguments for inspiral generator (RomanAmplitude)
amplitude_kwargs = {
    "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    "use_gpu": use_gpu  # GPU is available in this class
}

# keyword arguments for Ylm generator (GetYlms)
Ylm_kwargs = {
    "assume_positive_m": False  # if we assume positive m, it will generate negative m for all m>0
}

# keyword arguments for summation generator (InterpolatedModeSum)
sum_kwargs = {
    "use_gpu": use_gpu,  # GPU is availabel for this type of summation
    "pad_output": False,
}

from few.waveform import FastSchwarzschildEccentricFlux, SlowSchwarzschildEccentricFlux, GenerateEMRIWaveform

few = FastSchwarzschildEccentricFlux(
    inspiral_kwargs=inspiral_kwargs,
    amplitude_kwargs=amplitude_kwargs,
    Ylm_kwargs=Ylm_kwargs,
    sum_kwargs=sum_kwargs,
    use_gpu=use_gpu,
)

In [3]:
import numpy as np
import matplotlib.pyplot as plt



In [4]:
import cupy as cp 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp
# Define parameters
M = 4e6
mu = 5e-2
dist1 = 0.0000078705
e0 = 0.0
theta = np.pi / 2  # polar viewing angle
phi = 0  # azimuthal viewing angle
dt = 15
r = cp.linspace(10, 16, 201)

# Create a list to store the SNR profile
snrProfile = []

# Define the function to simulate the gravitational wave and calculate SNR
def calculate_snr(p0, L):
    wave = few(M, mu, p0, e0, theta, phi, dist=dist1, dt=dt, T=L)
    fft_real = cp.fft.fft(wave.real)
    fft_imag = cp.fft.fft(wave.imag)
    power_spectrum_real = cp.abs(fft_real) ** 2
    power_spectrum_imag = cp.abs(fft_imag) ** 2
    power_spectrum = power_spectrum_real + power_spectrum_imag
    sum_power = cp.sqrt(cp.sum(power_spectrum))
    return sum_power

# Iterate over different values of T
for L in [1 / 365, 0.1, 0.5, 1]:
    snrProfile = []
    for p0 in r:
        snrProfile.append(calculate_snr(p0, L))
    plt.figure()
    plt.plot(cp.asnumpy(r), cp.asnumpy(snrProfile), '.')
    plt.xlabel('r')
    plt.title(f'SNR Profile for T = {L} years')
    plt.show()


RuntimeError: CuPy failed to load libnvrtc.so.11.2: OSError: libnvrtc.so.11.2: cannot open shared object file: No such file or directory